#  Complete Guide to Retrieval-Augmented Generation (RAG)
## From Zero to Production: A Comprehensive Tutorial

**Author:** Jaswanth Gurujala
**Course:** INFO 7390 - Advanced Data Science and Architecture  


---

##  What You'll Learn

By the end of this notebook, you will be able to:
1.  Understand what RAG is and why it matters
2.  Build a simple RAG system from scratch
3.  Implement advanced RAG techniques (chunking, hybrid search, re-ranking)
4.  Evaluate and optimize RAG performance
5.  Deploy a production-ready RAG application

---

##  Learning Path

```
Part 1: Foundations → Part 2: Simple RAG → Part 3: Advanced RAG → Part 4: Evaluation → Part 5: Production
```

---
# Part 1: Understanding RAG - The "Why" and "What"

##  The Problem RAG Solves

### Scenario: Imagine you're building a chatbot for your company

**Without RAG:**
- LLM only knows information from its training data (cutoff date)
- Can't access your company's internal documents
- Might hallucinate answers about your specific context
- Can't update without retraining (expensive!)

**With RAG:**
- LLM retrieves relevant information from your documents
- Grounds responses in actual company data
- Updates instantly when you add new documents
- Much cheaper than fine-tuning

---

##  RAG Architecture: The Big Picture

```
┌─────────────────────────────────────────────────────────────┐
│                     RAG SYSTEM                              │
│                                                             │
│  1. INDEXING (Offline)          2. RETRIEVAL (Runtime)     │
│  ┌─────────────────┐             ┌──────────────┐          │
│  │  Documents      │             │ User Query   │          │
│  └────────┬────────┘             └──────┬───────┘          │
│           │                              │                  │
│           ▼                              ▼                  │
│  ┌─────────────────┐             ┌──────────────┐          │
│  │ Text Splitting  │             │  Embedding   │          │
│  │  (Chunking)     │             │    Model     │          │
│  └────────┬────────┘             └──────┬───────┘          │
│           │                              │                  │
│           ▼                              ▼                  │
│  ┌─────────────────┐             ┌──────────────┐          │
│  │  Embedding      │             │   Search     │          │
│  │    Model        │             │  Vector DB   │          │
│  └────────┬────────┘             └──────┬───────┘          │
│           │                              │                  │
│           ▼                              ▼                  │
│  ┌─────────────────┐             ┌──────────────┐          │
│  │  Vector DB      │◄────────────│  Top-K Docs  │          │
│  │   (Storage)     │             └──────┬───────┘          │
│  └─────────────────┘                    │                  │
│                                          ▼                  │
│                              ┌────────────────────┐         │
│                              │  LLM Generation    │         │
│                              │ (Query + Context)  │         │
│                              └─────────┬──────────┘         │
│                                        │                    │
│                                        ▼                    │
│                              ┌────────────────────┐         │
│                              │   Final Answer     │         │
│                              └────────────────────┘         │
└─────────────────────────────────────────────────────────────┘
```

---

##  Key Concepts

### 1. **Embeddings**
Converting text into numerical vectors that capture semantic meaning
- Similar texts → Similar vectors
- Example: "dog" and "puppy" are close in vector space

### 2. **Vector Database**
Specialized database for storing and searching embeddings efficiently
- Examples: Pinecone, Chroma, FAISS, Weaviate

### 3. **Retrieval**
Finding the most relevant documents for a query
- Similarity search (cosine, dot product)
- Returns Top-K most relevant chunks

### 4. **Generation**
LLM uses retrieved context to generate an answer
- Context injection into prompt
- Grounded, fact-based responses

---

##  When to Use RAG vs. Alternatives

| Approach | Best For | Cost | Update Speed |
|----------|----------|------|-------------|
| **RAG** | External knowledge, frequent updates |  Low |  Instant |
| **Fine-tuning** | Style/tone adaptation, domain-specific language |  High |  Slow (retrain) |
| **Prompt Engineering** | Task-specific instructions |  Very Low |  Instant |
| **Pre-training** | General intelligence |  Extreme |  Very Slow |

**Use RAG when you need:**
- Up-to-date information
- Company/domain-specific knowledge
- Traceable, citeable sources
- Cost-effective solution

---
# Part 2: Building Your First RAG System

Let's build a simple but functional RAG system step-by-step!

##  Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai sentence-transformers chromadb langchain tiktoken numpy pandas matplotlib seaborn plotly python-dotenv

In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import re
from collections import defaultdict

# Embeddings and Vector Store
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

# LLM Integration
from openai import OpenAI

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Warnings
import warnings
warnings.filterwarnings('ignore')

print(" All libraries imported successfully!")

##  API Configuration


In [ ]:
# Option 1: Set API key directly (not recommended for production)
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

# Option 2: Load from .env file (recommended)
from dotenv import load_dotenv
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print(" API client initialized!")

---
##  Step 1: Create Sample Documents

Let's create a knowledge base about data science concepts from our course!

In [ ]:
# Sample documents about data science concepts
documents = [
    {
        "id": "doc1",
        "title": "GIGO Principle",
        "content": """The GIGO (Garbage In, Garbage Out) principle is fundamental to data science.
        It states that the quality of output is determined by the quality of input data.
        No matter how sophisticated your model is, poor quality data will lead to poor results.
        Data scientists must invest significant time in data validation, cleaning, and quality assessment
        before building models. This includes checking for missing values, outliers, inconsistencies,
        and ensuring data accurately represents the problem domain."""
    },
    {
        "id": "doc2",
        "title": "Causal Inference",
        "content": """Causal inference is the process of determining the independent, actual effect of a
        particular phenomenon that is a component of a larger system. Unlike correlation, which simply
        measures association, causation implies that one variable directly influences another.
        Common methods include randomized controlled trials (RCTs), propensity score matching,
        instrumental variables, and difference-in-differences analysis. These methods help us answer
        'what if' questions and understand the true impact of interventions."""
    },
    {
        "id": "doc3",
        "title": "Time Series Forecasting",
        "content": """Time series forecasting involves predicting future values based on historical data points
        collected over time. Key components include trend (long-term direction), seasonality (regular patterns),
        and noise (random fluctuations). Popular methods include ARIMA (AutoRegressive Integrated Moving Average),
        Prophet (developed by Facebook), and LSTM networks (Long Short-Term Memory neural networks).
        Applications range from stock price prediction to weather forecasting and demand planning."""
    },
    {
        "id": "doc4",
        "title": "A/B Testing",
        "content": """A/B testing is a randomized experiment comparing two or more variants to determine
        which performs better. It's crucial for making data-driven decisions in product development.
        Key considerations include sample size calculation, statistical significance, p-values,
        and avoiding peeking at results too early. Common pitfalls include selection bias,
        insufficient sample size, and not accounting for multiple testing. Modern approaches include
        multi-armed bandits and sequential testing for faster iteration."""
    },
    {
        "id": "doc5",
        "title": "Feature Engineering",
        "content": """Feature engineering is the process of creating new features from raw data to improve
        model performance. It's often more impactful than algorithm selection. Techniques include:
        polynomial features (creating interactions), encoding categorical variables (one-hot, target encoding),
        handling missing values (imputation strategies), scaling (normalization, standardization),
        and domain-specific transformations. Good feature engineering requires deep understanding
        of both the data and the problem domain. It's an iterative process that combines domain expertise
        with statistical techniques."""
    },
    {
        "id": "doc6",
        "title": "Model Evaluation Metrics",
        "content": """Choosing the right evaluation metric is crucial for model success. For classification:
        accuracy (overall correctness), precision (correctness of positive predictions), recall (coverage
        of actual positives), F1-score (harmonic mean of precision and recall), and ROC-AUC (discrimination ability).
        For regression: MSE (mean squared error), RMSE (root mean squared error), MAE (mean absolute error),
        and R-squared (proportion of variance explained). The choice depends on business context - for example,
        in fraud detection, recall might be more important than precision."""
    },
    {
        "id": "doc7",
        "title": "Deep Learning Fundamentals",
        "content": """Deep learning uses neural networks with multiple layers to learn hierarchical representations
        of data. Key architectures include: CNNs (Convolutional Neural Networks) for image processing,
        RNNs (Recurrent Neural Networks) for sequential data, Transformers for natural language processing,
        and GANs (Generative Adversarial Networks) for generation tasks. Training involves backpropagation,
        gradient descent optimization, and regularization techniques like dropout and batch normalization.
        Deep learning excels with large datasets but requires significant computational resources."""
    },
    {
        "id": "doc8",
        "title": "Bias and Fairness in ML",
        "content": """Machine learning models can perpetuate and amplify societal biases present in training data.
        Types of bias include historical bias (biased data collection), representation bias (unrepresentative samples),
        and measurement bias (flawed metrics). Fairness metrics include demographic parity (equal positive rates
        across groups), equalized odds (equal true/false positive rates), and individual fairness (similar individuals
        treated similarly). Mitigation strategies involve careful data collection, bias-aware algorithms,
        and continuous monitoring in production. Ethical AI requires ongoing vigilance and diverse perspectives."""
    }
]

print(f" Created {len(documents)} sample documents")
print(f"\nDocument titles:")
for doc in documents:
    print(f"  • {doc['title']}")

---
##  Step 2: Text Chunking

**Why chunk?**
- Documents are often too long for LLM context windows
- Smaller chunks provide more precise retrieval
- Balance: too small = loss of context, too large = less precision

**Common strategies:**
1. **Fixed-size chunking** - Simple, consistent
2. **Sentence-based** - Natural boundaries
3. **Paragraph-based** - Semantic units
4. **Semantic chunking** - ML-based meaningful splits

In [ ]:
class SimpleChunker:
    """
    A simple text chunker that splits documents into overlapping chunks.

    Parameters:
    -----------
    chunk_size : int
        Target size of each chunk in characters
    overlap : int
        Number of overlapping characters between chunks (helps maintain context)
    """

    def __init__(self, chunk_size: int = 500, overlap: int = 50):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def chunk_text(self, text: str, metadata: Dict = None) -> List[Dict]:
        """
        Split text into overlapping chunks.

        Returns:
        --------
        List of dictionaries containing chunk text and metadata
        """
        # Clean the text
        text = re.sub(r'\s+', ' ', text).strip()

        chunks = []
        start = 0
        chunk_id = 0

        while start < len(text):
            # Define chunk boundaries
            end = start + self.chunk_size

            # If this isn't the last chunk, try to break at sentence boundary
            if end < len(text):
                # Look for sentence ending near the chunk boundary
                last_period = text.rfind('.', start, end)
                if last_period != -1 and last_period > start + self.chunk_size * 0.5:
                    end = last_period + 1

            # Extract chunk
            chunk_text = text[start:end].strip()

            # Create chunk object with metadata
            chunk = {
                'text': chunk_text,
                'chunk_id': chunk_id,
                'start_pos': start,
                'end_pos': end,
                'metadata': metadata or {}
            }
            chunks.append(chunk)

            # Move to next chunk with overlap
            start = end - self.overlap
            chunk_id += 1

        return chunks


# Initialize chunker
chunker = SimpleChunker(chunk_size=400, overlap=50)

# Chunk all documents
all_chunks = []
for doc in documents:
    metadata = {
        'doc_id': doc['id'],
        'title': doc['title']
    }
    chunks = chunker.chunk_text(doc['content'], metadata)
    all_chunks.extend(chunks)

print(f" Created {len(all_chunks)} chunks from {len(documents)} documents")
print(f"\nExample chunk:")
print(f"Title: {all_chunks[0]['metadata']['title']}")
print(f"Text: {all_chunks[0]['text'][:200]}...")

###  Visualize Chunk Distribution

In [ ]:
# Analyze chunk sizes
chunk_lengths = [len(chunk['text']) for chunk in all_chunks]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(chunk_lengths, bins=20, color='skyblue', edgecolor='black')
axes[0].axvline(np.mean(chunk_lengths), color='red', linestyle='--', label=f'Mean: {np.mean(chunk_lengths):.0f}')
axes[0].set_xlabel('Chunk Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Chunk Lengths')
axes[0].legend()

# Box plot
axes[1].boxplot(chunk_lengths, vert=True)
axes[1].set_ylabel('Chunk Length (characters)')
axes[1].set_title('Chunk Length Statistics')
axes[1].grid(axis='y')

plt.tight_layout()
plt.show()

print(f"\n Chunk Statistics:")
print(f"  Mean length: {np.mean(chunk_lengths):.0f} characters")
print(f"  Median length: {np.median(chunk_lengths):.0f} characters")
print(f"  Min length: {min(chunk_lengths)} characters")
print(f"  Max length: {max(chunk_lengths)} characters")

---
##  Step 3: Generate Embeddings

**What are embeddings?**
- Numerical representations of text that capture semantic meaning
- Similar meanings → Similar vectors
- Typically 384 to 1536 dimensions

**Popular embedding models:**
- `all-MiniLM-L6-v2` - Fast, 384 dims, good for most tasks
- `all-mpnet-base-v2` - Better quality, 768 dims
- OpenAI `text-embedding-ada-002` - 1536 dims, commercial

In [ ]:
# Initialize embedding model
# Using sentence-transformers (open-source, runs locally)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

print(f" Loaded embedding model: all-MiniLM-L6-v2")
print(f"   Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")

In [ ]:
def generate_embeddings(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Generate embeddings for a list of texts.

    Parameters:
    -----------
    texts : List[str]
        List of text strings to embed
    batch_size : int
        Number of texts to process at once (for efficiency)

    Returns:
    --------
    numpy array of shape (n_texts, embedding_dim)
    """
    return embedding_model.encode(texts, batch_size=batch_size, show_progress_bar=True)


# Generate embeddings for all chunks
chunk_texts = [chunk['text'] for chunk in all_chunks]
embeddings = generate_embeddings(chunk_texts)

print(f"\n Generated embeddings for {len(chunk_texts)} chunks")
print(f"   Embedding shape: {embeddings.shape}")
print(f"   Memory usage: {embeddings.nbytes / 1024 / 1024:.2f} MB")

###  Understanding Embeddings: Similarity Example

In [ ]:
# Let's see how embeddings capture semantic similarity
from sklearn.metrics.pairwise import cosine_similarity

test_sentences = [
    "Machine learning is a subset of artificial intelligence",
    "Deep learning uses neural networks",
    "I love eating pizza for dinner",
    "Data science involves statistics and programming"
]

test_embeddings = embedding_model.encode(test_sentences)
similarity_matrix = cosine_similarity(test_embeddings)

# Visualize similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=[f"S{i+1}" for i in range(len(test_sentences))],
            yticklabels=[f"S{i+1}" for i in range(len(test_sentences))],
            cbar_kws={'label': 'Cosine Similarity'})
plt.title('Semantic Similarity Between Sentences', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTest Sentences:")
for i, sent in enumerate(test_sentences, 1):
    print(f"S{i}: {sent}")

print("\n💡 Observations:")
print("  • S1 and S2 (ML-related) have high similarity")
print("  • S3 (pizza) has low similarity with tech topics")
print("  • S4 (data science) is similar to S1 and S2")

---
##  Step 4: Store in Vector Database

**Why use a vector database?**
- Efficient similarity search at scale
- Optimized storage and indexing
- Fast nearest neighbor search

We'll use **ChromaDB** - a simple, lightweight vector database perfect for learning!

In [ ]:
# Initialize ChromaDB client
chroma_client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma_db"  # Where to store the database
))

# Create or get collection
collection_name = "data_science_docs"

# Delete if exists (for clean slate)
try:
    chroma_client.delete_collection(collection_name)
except:
    pass

# Create new collection
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"description": "Data Science course materials"}
)

print(f" Created ChromaDB collection: {collection_name}")

In [ ]:
# Add documents to collection
collection.add(
    documents=[chunk['text'] for chunk in all_chunks],
    embeddings=embeddings.tolist(),
    metadatas=[chunk['metadata'] for chunk in all_chunks],
    ids=[f"chunk_{i}" for i in range(len(all_chunks))]
)

print(f" Added {len(all_chunks)} chunks to vector database")
print(f"\n Collection stats:")
print(f"   Total documents: {collection.count()}")
print(f"   Embedding dimension: {len(embeddings[0])}")

---
##  Step 5: Implement Retrieval

Now let's retrieve relevant documents based on a query!

In [ ]:
def retrieve_documents(query: str, n_results: int = 3) -> Dict:
    """
    Retrieve the most relevant documents for a query.

    Parameters:
    -----------
    query : str
        User's question or search query
    n_results : int
        Number of top results to return

    Returns:
    --------
    Dictionary with retrieved documents and metadata
    """
    # Generate query embedding
    query_embedding = embedding_model.encode([query])[0]

    # Search in vector database
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )

    return results


# Test retrieval
test_query = "How do I evaluate a classification model?"
print(f" Query: {test_query}\n")

results = retrieve_documents(test_query, n_results=3)

print("\n📄 Retrieved Documents:\n")
for i, (doc, metadata, distance) in enumerate(zip(results['documents'][0],
                                                   results['metadatas'][0],
                                                   results['distances'][0]), 1):
    print(f"Result {i} (similarity: {1 - distance:.3f}):")
    print(f"Title: {metadata['title']}")
    print(f"Text: {doc[:200]}...\n")

---
##  Step 6: Generate Answers with LLM

Now we combine retrieval with generation - the heart of RAG!

In [ ]:
def generate_answer(query: str, n_results: int = 3, model: str = "gpt-3.5-turbo") -> Dict:
    """
    Generate an answer using RAG: Retrieve relevant docs + Generate with LLM.

    Parameters:
    -----------
    query : str
        User's question
    n_results : int
        Number of documents to retrieve
    model : str
        OpenAI model to use

    Returns:
    --------
    Dictionary with answer, sources, and metadata
    """
    # Step 1: Retrieve relevant documents
    results = retrieve_documents(query, n_results=n_results)

    # Step 2: Prepare context from retrieved documents
    context = "\n\n".join([
        f"Document {i+1} ({results['metadatas'][0][i]['title']}):\n{doc}"
        for i, doc in enumerate(results['documents'][0])
    ])

    # Step 3: Create prompt with context
    system_prompt = """You are a helpful data science tutor.
    Answer questions based on the provided context documents.
    If the context doesn't contain enough information, say so.
    Always cite which document(s) you used."""

    user_prompt = f"""Context:
{context}

Question: {query}

Please provide a clear, accurate answer based on the context above."""

    # Step 4: Generate answer with LLM
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )

    answer = response.choices[0].message.content

    # Step 5: Package results
    return {
        'query': query,
        'answer': answer,
        'sources': [
            {
                'title': metadata['title'],
                'text': doc[:200] + "...",
                'similarity': 1 - distance
            }
            for doc, metadata, distance in zip(results['documents'][0],
                                               results['metadatas'][0],
                                               results['distances'][0])
        ]
    }


# Test the RAG system!
test_questions = [
    "How do I evaluate a classification model?",
    "What is the GIGO principle and why does it matter?",
    "What's the difference between correlation and causation?"
]

for question in test_questions:
    print("=" * 80)
    print(f"\n❓ Question: {question}\n")

    result = generate_answer(question)

    print(f"💡 Answer:\n{result['answer']}\n")

    print(" Sources used:")
    for i, source in enumerate(result['sources'], 1):
        print(f"  {i}. {source['title']} (similarity: {source['similarity']:.3f})")

    print()

---
#  Congratulations! You've built a working RAG system!

##  What you've accomplished:
1.  Created and chunked a document knowledge base
2.  Generated semantic embeddings
3.  Stored embeddings in a vector database
4.  Implemented similarity search retrieval
5.  Combined retrieval with LLM generation

---

##  What's Next?

In the following sections, we'll explore:
- **Advanced chunking strategies**
- **Hybrid search** (keyword + semantic)
- **Re-ranking** for better relevance
- **Evaluation metrics** for RAG quality
- **Production considerations**

---
# Part 3: Advanced RAG Techniques

Now let's level up with advanced techniques that improve RAG performance!

##  Advanced Chunking Strategies

### Problem with Simple Chunking:
- May split in the middle of important context
- Doesn't respect semantic boundaries

### Better Approaches:
1. **Sentence-aware chunking** - Split at sentence boundaries
2. **Sliding window** - Overlapping chunks maintain context
3. **Semantic chunking** - Split based on topic changes

In [ ]:
class AdvancedChunker:
    """
    Advanced chunking with multiple strategies.
    """

    def __init__(self, chunk_size: int = 500, overlap: int = 100, method: str = 'sentence'):
        """
        Parameters:
        -----------
        chunk_size : int
            Target chunk size in characters
        overlap : int
            Overlap between chunks
        method : str
            Chunking method: 'fixed', 'sentence', or 'semantic'
        """
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.method = method

    def _split_into_sentences(self, text: str) -> List[str]:
        """Split text into sentences using regex."""
        # Simple sentence splitter (can be improved with NLTK or spaCy)
        sentences = re.split(r'(?<=[.!?])\s+', text)
        return [s.strip() for s in sentences if s.strip()]

    def chunk_by_sentences(self, text: str, metadata: Dict = None) -> List[Dict]:
        """
        Chunk text by grouping sentences up to chunk_size.
        Better respects semantic boundaries!
        """
        sentences = self._split_into_sentences(text)
        chunks = []
        current_chunk = []
        current_size = 0

        for sentence in sentences:
            sentence_len = len(sentence)

            # If adding this sentence exceeds chunk_size, save current chunk
            if current_size + sentence_len > self.chunk_size and current_chunk:
                chunk_text = ' '.join(current_chunk)
                chunks.append({
                    'text': chunk_text,
                    'chunk_id': len(chunks),
                    'metadata': metadata or {},
                    'num_sentences': len(current_chunk)
                })

                # Start new chunk with overlap (keep last sentence)
                if self.overlap > 0 and current_chunk:
                    current_chunk = [current_chunk[-1]]
                    current_size = len(current_chunk[0])
                else:
                    current_chunk = []
                    current_size = 0

            current_chunk.append(sentence)
            current_size += sentence_len

        # Add final chunk
        if current_chunk:
            chunks.append({
                'text': ' '.join(current_chunk),
                'chunk_id': len(chunks),
                'metadata': metadata or {},
                'num_sentences': len(current_chunk)
            })

        return chunks


# Compare chunking strategies
sample_text = documents[0]['content']

# Simple chunking
simple_chunker = SimpleChunker(chunk_size=300, overlap=50)
simple_chunks = simple_chunker.chunk_text(sample_text)

# Advanced sentence-based chunking
advanced_chunker = AdvancedChunker(chunk_size=300, overlap=1, method='sentence')
advanced_chunks = advanced_chunker.chunk_by_sentences(sample_text)

print("📊 Chunking Comparison:\n")
print(f"Simple Chunking:")
print(f"  • Number of chunks: {len(simple_chunks)}")
print(f"  • Average chunk length: {np.mean([len(c['text']) for c in simple_chunks]):.0f} chars\n")

print(f"Sentence-Based Chunking:")
print(f"  • Number of chunks: {len(advanced_chunks)}")
print(f"  • Average chunk length: {np.mean([len(c['text']) for c in advanced_chunks]):.0f} chars")
print(f"  • Average sentences per chunk: {np.mean([c['num_sentences'] for c in advanced_chunks]):.1f}")

print("\n💡 Sentence-based chunking respects natural boundaries!")

---
##  Hybrid Search: Combining Semantic + Keyword Search

**Why hybrid?**
- Semantic search: Great for meaning and context
- Keyword search (BM25): Better for exact terms and proper nouns
- **Hybrid = Best of both worlds!**

Example:
- Query: "ARIMA model" → Keyword search finds exact "ARIMA" mentions
- Query: "How to predict future values" → Semantic search understands intent

In [ ]:
from rank_bm25 import BM25Okapi
import nltk
# Download required NLTK data (run once)
# nltk.download('punkt')

class HybridRetriever:
    """
    Hybrid retrieval combining semantic (embedding) and keyword (BM25) search.
    """

    def __init__(self, documents: List[str], embeddings: np.ndarray, alpha: float = 0.5):
        """
        Parameters:
        -----------
        documents : List[str]
            List of document texts
        embeddings : np.ndarray
            Document embeddings
        alpha : float
            Weight for semantic search (1-alpha for keyword search)
            0.5 = equal weight, 0.7 = favor semantic, 0.3 = favor keyword
        """
        self.documents = documents
        self.embeddings = embeddings
        self.alpha = alpha

        # Initialize BM25
        tokenized_docs = [doc.lower().split() for doc in documents]
        self.bm25 = BM25Okapi(tokenized_docs)

    def search(self, query: str, n_results: int = 5) -> List[Tuple[int, float, str]]:
        """
        Perform hybrid search.

        Returns:
        --------
        List of (index, score, source) tuples
        """
        # 1. Semantic search scores
        query_embedding = embedding_model.encode([query])[0]
        semantic_scores = cosine_similarity([query_embedding], self.embeddings)[0]

        # 2. Keyword search scores (BM25)
        tokenized_query = query.lower().split()
        keyword_scores = self.bm25.get_scores(tokenized_query)

        # 3. Normalize scores to [0, 1]
        semantic_scores_norm = (semantic_scores - semantic_scores.min()) / (semantic_scores.max() - semantic_scores.min() + 1e-10)
        keyword_scores_norm = (keyword_scores - keyword_scores.min()) / (keyword_scores.max() - keyword_scores.min() + 1e-10)

        # 4. Combine scores
        hybrid_scores = self.alpha * semantic_scores_norm + (1 - self.alpha) * keyword_scores_norm

        # 5. Get top results
        top_indices = np.argsort(hybrid_scores)[::-1][:n_results]

        results = [
            (idx, hybrid_scores[idx], f"semantic: {semantic_scores_norm[idx]:.3f}, keyword: {keyword_scores_norm[idx]:.3f}")
            for idx in top_indices
        ]

        return results


# Test hybrid search
hybrid_retriever = HybridRetriever(
    documents=chunk_texts,
    embeddings=embeddings,
    alpha=0.6  # 60% semantic, 40% keyword
)

# Test queries
test_queries = [
    "ARIMA forecasting method",  # Should benefit from keyword search
    "How to predict future values in time series"  # Should benefit from semantic search
]

for query in test_queries:
    print(f"\n🔍 Query: {query}")
    print("="*80)

    results = hybrid_retriever.search(query, n_results=3)

    for i, (idx, score, breakdown) in enumerate(results, 1):
        print(f"\nResult {i} (score: {score:.3f}):")
        print(f"  {breakdown}")
        print(f"  Title: {all_chunks[idx]['metadata']['title']}")
        print(f"  Text: {chunk_texts[idx][:150]}...")

---
##  Re-Ranking: Improving Retrieval Quality

**The Problem:**
- Initial retrieval might return relevant but not *optimal* results
- Order matters! LLMs work best with most relevant context first

**Solution: Re-ranking**
1. Retrieve broader set (e.g., top 20)
2. Use a specialized re-ranker model
3. Return top-K after re-ranking

**Popular re-rankers:**
- Cross-encoders (more accurate but slower)
- LLM-based re-ranking
- Custom trained models

In [ ]:
from sentence_transformers import CrossEncoder

class ReRanker:
    """
    Re-rank retrieved documents using a cross-encoder model.
    """

    def __init__(self, model_name: str = 'cross-encoder/ms-marco-MiniLM-L-6-v2'):
        """
        Initialize re-ranker with a cross-encoder model.
        This model directly scores query-document pairs.
        """
        self.model = CrossEncoder(model_name)

    def rerank(self, query: str, documents: List[str], top_k: int = 3) -> List[Tuple[int, float]]:
        """
        Re-rank documents for a query.

        Returns:
        --------
        List of (original_index, rerank_score) tuples, sorted by score
        """
        # Score all query-document pairs
        pairs = [[query, doc] for doc in documents]
        scores = self.model.predict(pairs)

        # Sort by score and return top-k
        ranked_results = sorted(
            enumerate(scores),
            key=lambda x: x[1],
            reverse=True
        )[:top_k]

        return ranked_results


# Initialize re-ranker
reranker = ReRanker()

# Compare: Retrieval vs. Retrieval + Re-ranking
query = "What metrics should I use to evaluate my machine learning model?"

print(f"🔍 Query: {query}\n")

# Step 1: Initial retrieval (get more candidates)
initial_results = retrieve_documents(query, n_results=10)
candidate_docs = initial_results['documents'][0]

print(" Top 3 WITHOUT Re-ranking:")
for i in range(3):
    print(f"\n{i+1}. {initial_results['metadatas'][0][i]['title']}")
    print(f"   Similarity: {1 - initial_results['distances'][0][i]:.3f}")
    print(f"   {candidate_docs[i][:150]}...")

# Step 2: Re-rank
reranked = reranker.rerank(query, candidate_docs, top_k=3)

print("\n\n🎯 Top 3 WITH Re-ranking:")
for rank, (orig_idx, score) in enumerate(reranked, 1):
    print(f"\n{rank}. {initial_results['metadatas'][0][orig_idx]['title']}")
    print(f"   Re-rank score: {score:.3f}")
    print(f"   {candidate_docs[orig_idx][:150]}...")

print("\n\n💡 Notice how re-ranking can change the order to be more relevant!")

---
##  Query Enhancement Techniques

**Problem:** User queries are often ambiguous or poorly formulated

**Solutions:**
1. **Query expansion** - Add related terms
2. **Query rewriting** - Reformulate for better retrieval
3. **Multi-query** - Generate multiple perspectives
4. **HyDE** (Hypothetical Document Embeddings) - Generate ideal answer, search with that

In [ ]:
def expand_query_with_llm(query: str) -> List[str]:
    """
    Generate multiple reformulations of a query using LLM.
    This helps capture different aspects and improves recall!
    """
    prompt = f"""Given this user question, generate 3 different ways to phrase it
    that would help retrieve relevant information. Make them diverse.

    Original question: {query}

    Provide 3 reformulations, one per line, without numbering:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )

    reformulations = response.choices[0].message.content.strip().split('\n')
    reformulations = [r.strip() for r in reformulations if r.strip()]

    return [query] + reformulations[:3]  # Original + 3 variations


def multi_query_retrieval(query: str, n_results_per_query: int = 3) -> Dict:
    """
    Retrieve documents using multiple query formulations and merge results.
    """
    # Generate query variations
    queries = expand_query_with_llm(query)

    print("🔄 Generated query variations:")
    for i, q in enumerate(queries, 1):
        print(f"  {i}. {q}")

    # Retrieve for each query variation
    all_results = []
    seen_docs = set()

    for q in queries:
        results = retrieve_documents(q, n_results=n_results_per_query)

        for doc, metadata, distance in zip(results['documents'][0],
                                           results['metadatas'][0],
                                           results['distances'][0]):
            doc_id = metadata['title'] + doc[:50]  # Simple deduplication
            if doc_id not in seen_docs:
                all_results.append({
                    'document': doc,
                    'metadata': metadata,
                    'score': 1 - distance
                })
                seen_docs.add(doc_id)

    # Sort by score and return top results
    all_results.sort(key=lambda x: x['score'], reverse=True)

    return {
        'original_query': query,
        'query_variations': queries,
        'results': all_results[:n_results_per_query * 2]  # Return more diverse results
    }


# Test multi-query retrieval
test_query = "How do I check if my data is good?"

print(f"\n❓ Original Query: {test_query}\n")
print("="*80)

multi_results = multi_query_retrieval(test_query)

print("\n\n📄 Top Retrieved Documents:")
for i, result in enumerate(multi_results['results'][:3], 1):
    print(f"\n{i}. {result['metadata']['title']} (score: {result['score']:.3f})")
    print(f"   {result['document'][:200]}...")

---
# Part 4: RAG Evaluation & Optimization

**Critical Question:** How do we know if our RAG system is working well?

We need to evaluate both:
1. **Retrieval quality** - Are we finding the right documents?
2. **Generation quality** - Are the answers accurate and helpful?

##  Retrieval Metrics

Key metrics for evaluating retrieval:

1. **Precision@K** - Of the K retrieved docs, how many are relevant?
2. **Recall@K** - Of all relevant docs, how many did we retrieve?
3. **MRR (Mean Reciprocal Rank)** - Position of first relevant doc
4. **NDCG (Normalized Discounted Cumulative Gain)** - Considers ranking quality

In [ ]:
# Create evaluation dataset
evaluation_questions = [
    {
        "question": "How do I evaluate a classification model?",
        "relevant_docs": ["Model Evaluation Metrics"],  # Ground truth
        "category": "metrics"
    },
    {
        "question": "What is GIGO and why does it matter?",
        "relevant_docs": ["GIGO Principle"],
        "category": "data_quality"
    },
    {
        "question": "How can I predict future sales?",
        "relevant_docs": ["Time Series Forecasting"],
        "category": "forecasting"
    },
    {
        "question": "What's the difference between correlation and causation?",
        "relevant_docs": ["Causal Inference"],
        "category": "causality"
    },
    {
        "question": "How do I create better features for my model?",
        "relevant_docs": ["Feature Engineering"],
        "category": "feature_eng"
    }
]

def calculate_precision_at_k(retrieved_docs: List[str], relevant_docs: List[str], k: int) -> float:
    """Calculate Precision@K"""
    retrieved_k = retrieved_docs[:k]
    relevant_count = sum(1 for doc in retrieved_k if doc in relevant_docs)
    return relevant_count / k if k > 0 else 0

def calculate_recall_at_k(retrieved_docs: List[str], relevant_docs: List[str], k: int) -> float:
    """Calculate Recall@K"""
    retrieved_k = retrieved_docs[:k]
    relevant_count = sum(1 for doc in retrieved_k if doc in relevant_docs)
    return relevant_count / len(relevant_docs) if len(relevant_docs) > 0 else 0

def calculate_mrr(retrieved_docs: List[str], relevant_docs: List[str]) -> float:
    """Calculate Mean Reciprocal Rank"""
    for i, doc in enumerate(retrieved_docs, 1):
        if doc in relevant_docs:
            return 1 / i
    return 0

# Evaluate retrieval performance
results = []
k_values = [1, 3, 5]

for eval_item in evaluation_questions:
    question = eval_item['question']
    relevant_docs = eval_item['relevant_docs']

    # Retrieve documents
    retrieval_results = retrieve_documents(question, n_results=5)
    retrieved_titles = [meta['title'] for meta in retrieval_results['metadatas'][0]]

    # Calculate metrics
    metrics = {
        'question': question,
        'category': eval_item['category']
    }

    for k in k_values:
        metrics[f'precision@{k}'] = calculate_precision_at_k(retrieved_titles, relevant_docs, k)
        metrics[f'recall@{k}'] = calculate_recall_at_k(retrieved_titles, relevant_docs, k)

    metrics['mrr'] = calculate_mrr(retrieved_titles, relevant_docs)
    results.append(metrics)

# Create DataFrame for analysis
eval_df = pd.DataFrame(results)

print(" Retrieval Evaluation Results:\n")
print(eval_df[['question', 'precision@3', 'recall@3', 'mrr']].to_string(index=False))

print("\n\n📈 Average Metrics:")
for k in k_values:
    avg_precision = eval_df[f'precision@{k}'].mean()
    avg_recall = eval_df[f'recall@{k}'].mean()
    print(f"  Precision@{k}: {avg_precision:.3f}")
    print(f"  Recall@{k}: {avg_recall:.3f}")
print(f"\n  Mean Reciprocal Rank: {eval_df['mrr'].mean():.3f}")

###  Visualize Retrieval Performance

In [ ]:
# Plot metrics by category
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precision and Recall at different K values
metric_cols = [col for col in eval_df.columns if 'precision' in col or 'recall' in col]
eval_df[metric_cols].mean().plot(kind='bar', ax=axes[0], color=['skyblue', 'lightcoral', 'lightgreen', 'orange', 'purple', 'pink'])
axes[0].set_title('Average Precision and Recall @K', fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_xlabel('Metric')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].grid(axis='y', alpha=0.3)

# MRR by category
category_mrr = eval_df.groupby('category')['mrr'].mean().sort_values(ascending=False)
category_mrr.plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Mean Reciprocal Rank by Category', fontweight='bold')
axes[1].set_xlabel('MRR Score')
axes[1].set_ylabel('Category')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---
##  End-to-End RAG Evaluation

Now let's evaluate the complete RAG system (retrieval + generation):

**Key metrics:**
1. **Faithfulness** - Is the answer grounded in retrieved context?
2. **Answer Relevance** - Does it address the question?
3. **Context Relevance** - Are retrieved docs relevant?
4. **Correctness** - Is the answer factually correct?

In [ ]:
def evaluate_rag_answer(question: str, answer: str, context: str) -> Dict:
    """
    Evaluate a RAG answer using LLM-as-judge approach.
    """
    evaluation_prompt = f"""Evaluate this RAG system answer on a scale of 1-5 for each criterion:

Question: {question}

Retrieved Context:
{context}

Generated Answer:
{answer}

Rate the following (1=Poor, 5=Excellent):
1. Faithfulness: Is the answer based on the provided context (not hallucinated)?
2. Relevance: Does the answer directly address the question?
3. Completeness: Is the answer comprehensive enough?
4. Clarity: Is the answer clear and well-explained?

Provide ratings in this exact format:
Faithfulness: X
Relevance: X
Completeness: X
Clarity: X
Overall: X
Reasoning: [brief explanation]"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": evaluation_prompt}],
        temperature=0.3
    )

    eval_text = response.choices[0].message.content

    # Parse scores
    scores = {}
    for line in eval_text.split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip().lower()
            if key in ['faithfulness', 'relevance', 'completeness', 'clarity', 'overall']:
                try:
                    scores[key] = int(value.strip().split()[0])
                except:
                    pass

    return scores


# Evaluate RAG system end-to-end
print("🔍 Evaluating RAG System End-to-End...\n")

rag_evaluations = []

for eval_item in evaluation_questions[:3]:  # Test first 3
    question = eval_item['question']

    # Generate answer
    result = generate_answer(question, n_results=3)

    # Prepare context
    context = "\n\n".join([source['text'] for source in result['sources']])

    # Evaluate
    scores = evaluate_rag_answer(question, result['answer'], context)

    rag_evaluations.append({
        'question': question,
        **scores
    })

    print(f"Question: {question}")
    print(f"Scores: {scores}\n")

# Summary
eval_df = pd.DataFrame(rag_evaluations)
print("\n📊 Average Scores:")
for col in ['faithfulness', 'relevance', 'completeness', 'clarity', 'overall']:
    if col in eval_df.columns:
        print(f"  {col.capitalize()}: {eval_df[col].mean():.2f}/5")

---
# Part 5: Production Considerations & Best Practices

Moving from prototype to production RAG system

##  Optimization Strategies

### 1. **Caching**
- Cache embeddings (don't recompute for same texts)
- Cache LLM responses for common queries

### 2. **Batch Processing**
- Generate embeddings in batches
- Use async calls for LLM generation

### 3. **Index Optimization**
- Use approximate nearest neighbor (ANN) algorithms
- Consider HNSW, IVF indexes for large scale

### 4. **Cost Optimization**
- Use smaller embedding models when possible
- Implement retrieval thresholds
- Monitor token usage

In [ ]:
class ProductionRAGSystem:
    """
    Production-ready RAG system with optimization and monitoring.
    """

    def __init__(self, collection_name: str, embedding_model_name: str = 'all-MiniLM-L6-v2'):
        self.collection_name = collection_name
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.chroma_client = chromadb.Client()

        # Metrics tracking
        self.metrics = {
            'queries': 0,
            'total_latency': 0,
            'retrieval_latency': 0,
            'generation_latency': 0,
            'errors': 0
        }

        # Simple cache
        self.response_cache = {}
        self.cache_hits = 0

    def query(self, question: str, use_cache: bool = True, n_results: int = 3) -> Dict:
        """
        Query the RAG system with caching and monitoring.
        """
        import time
        start_time = time.time()

        try:
            # Check cache
            if use_cache and question in self.response_cache:
                self.cache_hits += 1
                return self.response_cache[question]

            # Retrieval
            retrieval_start = time.time()
            results = retrieve_documents(question, n_results=n_results)
            retrieval_time = time.time() - retrieval_start

            # Generation
            generation_start = time.time()
            answer = generate_answer(question, n_results=n_results)
            generation_time = time.time() - generation_start

            # Update metrics
            total_time = time.time() - start_time
            self.metrics['queries'] += 1
            self.metrics['total_latency'] += total_time
            self.metrics['retrieval_latency'] += retrieval_time
            self.metrics['generation_latency'] += generation_time

            # Cache result
            if use_cache:
                self.response_cache[question] = answer

            # Add timing info
            answer['timing'] = {
                'total': total_time,
                'retrieval': retrieval_time,
                'generation': generation_time
            }

            return answer

        except Exception as e:
            self.metrics['errors'] += 1
            return {'error': str(e)}

    def get_metrics(self) -> Dict:
        """Get performance metrics."""
        n_queries = max(self.metrics['queries'], 1)
        return {
            'total_queries': self.metrics['queries'],
            'cache_hit_rate': self.cache_hits / n_queries if n_queries > 0 else 0,
            'avg_total_latency': self.metrics['total_latency'] / n_queries,
            'avg_retrieval_latency': self.metrics['retrieval_latency'] / n_queries,
            'avg_generation_latency': self.metrics['generation_latency'] / n_queries,
            'error_rate': self.metrics['errors'] / n_queries if n_queries > 0 else 0
        }


# Test production system
prod_rag = ProductionRAGSystem(collection_name="data_science_docs")

# Run queries
test_queries = [
    "What is the GIGO principle?",
    "How do I evaluate my model?",
    "What is the GIGO principle?",  # Duplicate - should hit cache
]

print(" Testing Production RAG System...\n")

for query in test_queries:
    result = prod_rag.query(query)
    print(f"Query: {query}")
    print(f"  Latency: {result.get('timing', {}).get('total', 0):.3f}s")
    print()

# Print metrics
print("\n System Metrics:")
metrics = prod_rag.get_metrics()
for key, value in metrics.items():
    if 'latency' in key:
        print(f"  {key}: {value:.3f}s")
    elif 'rate' in key:
        print(f"  {key}: {value:.1%}")
    else:
        print(f"  {key}: {value}")

---
##  Common Pitfalls & Solutions

### Problem 1: **Context Length Exceeded**
**Symptoms:** Error when retrieved context + query > model's context window

**Solutions:**
- Reduce number of retrieved documents
- Truncate documents intelligently
- Use models with larger context windows

### Problem 2: **Poor Retrieval Quality**
**Symptoms:** Irrelevant documents retrieved

**Solutions:**
- Improve chunking strategy
- Try different embedding models
- Implement hybrid search
- Add metadata filtering

### Problem 3: **Hallucinations**
**Symptoms:** LLM generates information not in context

**Solutions:**
- Strengthen system prompt ("only use provided context")
- Lower temperature
- Implement fact-checking
- Add citations/sources

### Problem 4: **Slow Response Times**
**Symptoms:** High latency

**Solutions:**
- Implement caching
- Use faster embedding models
- Optimize vector database indexes
- Consider async processing

---
#  Exercises & Practice

## Exercise 1: Improve Chunking (Beginner)
**Task:** Modify the chunking strategy to preserve paragraph boundaries

**Hints:**
- Look for double newlines (`\n\n`)
- Try to keep complete paragraphs together
- Compare performance with original chunking

In [ ]:
# YOUR CODE HERE
# Implement paragraph-aware chunking

def paragraph_aware_chunking(text: str, max_chunk_size: int = 500) -> List[str]:
    """
    TODO: Implement chunking that respects paragraph boundaries.
    """
    pass

# Test your implementation

## Exercise 2: Add Metadata Filtering (Intermediate)
**Task:** Extend the retrieval system to filter by metadata (e.g., document category, date)

**Requirements:**
- Add category metadata to documents
- Implement filtered retrieval
- Test with queries like "Find forecasting methods"

In [ ]:
# YOUR CODE HERE
# Implement metadata filtering

def retrieve_with_filter(query: str, category: str = None, n_results: int = 3):
    """
    TODO: Add metadata filtering to retrieval.
    """
    pass

## Exercise 3: Build a Conversational RAG (Advanced)
**Task:** Extend the RAG system to handle multi-turn conversations

**Requirements:**
- Maintain conversation history
- Use history to contextualize queries
- Handle follow-up questions ("Tell me more about that")

**Bonus:** Implement conversation summarization to stay within context limits

In [ ]:
# YOUR CODE HERE
# Implement conversational RAG

class ConversationalRAG:
    """
    TODO: Implement RAG system with conversation memory.
    """
    def __init__(self):
        self.conversation_history = []

    def chat(self, user_input: str) -> str:
        pass

---
#  Project Ideas for Extension

## 1. **Domain-Specific RAG**
Build a RAG system for a specific domain:
- Medical literature assistant
- Legal document analyzer
- Code documentation helper

## 2. **Multi-Modal RAG**
Extend to handle images, tables, and structured data:
- Extract text from PDFs
- Index tables and figures
- Image caption generation

## 3. **RAG-Powered Chatbot**
Build a production chatbot with:
- Web interface (Streamlit/Gradio)
- User feedback collection
- Continuous learning

## 4. **Evaluation Suite**
Comprehensive evaluation framework:
- Automated test dataset generation
- Multiple evaluation metrics
- A/B testing different RAG configurations

## 5. **RAG + Fine-tuning**
Combine approaches:
- Use RAG for knowledge
- Fine-tune for domain language/style
- Compare hybrid vs. pure approaches

---
#  Additional Resources

## Papers & Blogs
- **RAG Paper:** "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks" (Lewis et al., 2020)
- **LangChain RAG Guide:** https://python.langchain.com/docs/use_cases/question_answering/
- **Pinecone Learning Center:** https://www.pinecone.io/learn/

## Tools & Libraries
- **LangChain:** RAG orchestration framework
- **LlamaIndex:** Data framework for LLM applications
- **ChromaDB:** Vector database
- **Weaviate:** Production vector database
- **RAGAS:** RAG evaluation framework

## Advanced Topics
- **RAGAs Framework:** Automated RAG evaluation
- **LLM Routing:** Directing queries to best source
- **Agentic RAG:** Using agents for complex retrieval
- **GraphRAG:** Using knowledge graphs with RAG

---
#  Conclusion

## What You've Learned

 **Foundations:** Understanding RAG architecture and components  
 **Implementation:** Building a working RAG system from scratch  
 **Advanced Techniques:** Hybrid search, re-ranking, query enhancement  
 **Evaluation:** Measuring and improving RAG performance  
 **Production:** Optimization and best practices  

## Key Takeaways

1. **RAG is powerful but not magic** - Quality depends on your data and configuration
2. **Chunking matters** - It's often the most impactful design decision
3. **Always evaluate** - Use metrics to guide improvements
4. **Start simple** - Add complexity only when needed
5. **Monitor in production** - Track performance and user satisfaction

## Next Steps

1. Complete the exercises in this notebook
2. Try the project ideas with your own data
3. Experiment with different configurations
4. Build something and share it!

---

**Remember:** The best way to learn is by doing. Start building!

**Questions?** Feel free to reach out or check the resources section.

